# Abstract


## Modelado Predictivo de la Satisfacción de Pasajeros en Aerolíneas Low-Cost: Un Enfoque de Machine Learning para Reseñas de Clientes de Ryanair

**Contexto:** La industria de aerolíneas de bajo costo opera en un equilibrio delicado entre precios competitivos y calidad de servicio, haciendo de la satisfacción del pasajero un determinante crítico del éxito empresarial. Ryanair, como la mayor aerolínea low-cost de Europa, presenta un caso de estudio compelling debido a su modelo de negocio distintivo y percepciones polarizadas de los clientes.

**Objetivo de Investigación:** Este estudio desarrolla un framework predictivo integral para analizar los determinantes del comportamiento de recomendación de pasajeros, integrando métricas estructuradas de rating con feedback textual no estructurado. La investigación aborda una brecha significativa en la literatura de aviación mediante el empleo de técnicas avanzadas de machine learning para decodificar la compleja interacción entre atributos de servicio, demographics de pasajeros y patrones lingüísticos en la formación de satisfacción.

**Metodología:** Utilizando un dataset de 276 reseñas detalladas de pasajeros, implementamos un enfoque analítico multimodal que combina:
- Análisis tradicional de ratings (Comodidad del Asiento, Servicio de Tripulación, Valor por Dinero, etc.)
- Rendimiento comparativo de tres algoritmos de clasificación: Regresión Logística, Random Forest y XGBoost


## Hipótesis Extendidas para Investigación

**Hipótesis 1: "Hipótesis de Primacía del Valor Económico"**
La percepción de valor por dinero supera a todos los demás atributos de servicio en la determinación de la satisfacción general y la probabilidad de recomendación en el contexto de aerolíneas low-cost.

**Hipótesis 2:  "Hipótesis Temporal"**
**"La satisfacción ha mejorado/deteriorado consistentemente en el tiempo"**
- *Variable*: `date_flown` o `date_published`
- Análisis de tendencias temporales

**Hipótesis 3: "Hipótesis de Vulnerabilidad del Segmento de Pasajeros"**
Ciertos segmentos de pasajeros (particularmente familias y parejas de leisure) demuestran sistemáticamente menor satisfacción debido expectativas desalineadas del servicio y sensibilidad a cargos adicionales.

**Conclusion de trabajo a realizar**
Este estudio desarrolla un modelo de clasificación para predecir la satisfacción de pasajeros con Ryanair basado en reseñas online, combinando análisis de texto (comentarios) con métricas estructuradas de evaluación. 



**Estructura del dataset:**
- **date_published**: Fecha de publicación de la reseña
- **overall_rating**: Calificación general (1-10)
- **passenger_country**: País del pasajero
- **trip_verified**: Si el viaje fue verificado
- **aircraft**: Tipo de avión (principalmente Boeing 737 variantes)
- **type_of_traveller**: Tipo de viajero (Leisure, Business, etc.)
- **seat_type**: Clase del asiento
- **origin/destination**: Aeropuertos de origen y destino
- **date_flown**: Fecha del vuelo
- **Métricas de evaluación**: seat_comfort, cabin_staff_service, food_&_beverages, ground_service, value_for_money, inflight_entertainment, wifi_&_connectivity
- **recommended**: Si recomendaría la aerolínea (yes/no)

**Observaciones iniciales:**
- Hay aproximadamente 900 reseñas
- Las calificaciones van desde 1.0 (muy malas) hasta 10.0 (excelentes)
- La mayoría de los vuelos son en clase económica
- Los pasajeros provienen de diversos países
- Los destinos cubren principalmente Europa

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [ ]:
# Cargar el dataset
file_path = "./dataset/ryanair_reviews.csv"

df = pd.read_csv(file_path)

In [ ]:

# Mostrar información general del dataset
print("Shape")
df.shape


In [ ]:

print("Columns")
df.columns


In [ ]:
df.info()

_En base de algunos tipos de datos, eliminaremos las columnas que no tienen sentido para la recomendacion, por ejemplo el no uso de comentaros o data del tipo NLP_

In [ ]:
columns_to_drop = [
    "id_review",
    "comment_title",
    "comment"
]

target = ['recommended']

In [ ]:
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace("(", "").str.replace(")", "")

In [ ]:
df.drop(columns=columns_to_drop, inplace=True)

In [ ]:
def analyze_missing_values(df):

    # Calcular valores nulos
    missing_values = df.isna().sum()
    missing_percentage = ((df.isna().sum() / len(df)) * 100).round(2)

    # Crear DataFrame
    missing_df = pd.DataFrame({
        "Cantidad_Nulos": missing_values,
        "Porcentaje_Nulos": missing_percentage
    })

    # Ordenar por porcentaje de nulos
    missing_df = missing_df.sort_values(by="Porcentaje_Nulos", ascending=False)    
    return missing_df

In [ ]:
missing_df = analyze_missing_values(df)
missing_df

In [ ]:
columns_high_missing_amount = missing_df.loc[missing_df.Porcentaje_Nulos>=40].index
columns_medium_missing_amount = missing_df.loc[(missing_df.Porcentaje_Nulos<40) & (missing_df.Porcentaje_Nulos>4)].index


In [ ]:
df["value_for_money"].value_counts(dropna=False,normalize=True)

_Para este caso muy puntual tiene la posibilidad de drop rows --> por la cantidad de datos presentados_

In [ ]:
df.dropna(subset="value_for_money", inplace=True)

In [ ]:
value_to_drop = ["cabin_staff_service","seat_comfort"]

In [ ]:
df.dropna(subset=value_to_drop, inplace=True)

_Eliminacion de duplicados_

In [ ]:
def remove_duplicates_with_info(df, subset=None, keep='first'):
    print("INFORMACIÓN DE ENTRADA:")
    print(f"   - Dimensiones: {df.shape[0]} filas × {df.shape[1]} columnas")
    print(f"   - Duplicados totales: {df.duplicated(subset=subset).sum()}")
    
    if subset:
        print(f"   - Verificando duplicados en columnas: {subset}")
    
    original_rows = df.shape[0]
    
    df_clean = df.drop_duplicates(subset=subset, keep=keep)
    
    print("\n INFORMACIÓN DE SALIDA:")
    print(f"   - Dimensiones: {df_clean.shape[0]} filas × {df_clean.shape[1]} columnas")
    print(f"   - Duplicados restantes: {df_clean.duplicated(subset=subset).sum()}")
    print(f"   - Filas eliminadas: {original_rows - df_clean.shape[0]}")
    print(f"   - Reducción: {((original_rows - df_clean.shape[0]) / original_rows * 100):.1f}%")
    
    return df_clean

In [ ]:
df = remove_duplicates_with_info(df)

In [ ]:
def quick_unique_count(df, columns=None):

    if columns is None:
        columns = df.columns
    
    print("CONTEO DE VALORES ÚNICOS POR COLUMNA")

    for col in columns:
        if col in df.columns:

            unique_count = df[col].nunique()
            total_count = len(df[col])
            null_count = df[col].isnull().sum()
            moda_value = df[col].mode()


            print(f" {col}:")
            print(f"   • Tipo: {df[col].dtype}")
            print(f"   • Valores únicos: {unique_count:,}")
            print(f"   • Valor de la moda: {moda_value}")
            print(f"   • Total valores: {total_count:,}")
            print(f"   • Valores nulos: {null_count:,}")
            print(f"   % Únicos: {(unique_count/total_count*100):.1f}%")
            
            # Mostrar algunos valores únicos (especial manejo para floats)
            if unique_count <= 10:
                unique_vals = df[col].dropna().unique()
                print(f"   • Valores: {sorted(unique_vals)}")
            elif df[col].dtype in ['float64', 'float32']:
                # Para floats, mostrar rango
                min_val = df[col].min()
                max_val = df[col].max()
                print(f"   • Rango: [{min_val:.2f} - {max_val:.2f}]")
            else:
                sample_vals = df[col].dropna().unique()[:3]
                print(f"   • Ejemplos: {list(sample_vals)}")
            
            print("-" * 40)


In [ ]:
from numpy import pad


def plot_by_dtype_subplots(df, columns=None):
    if columns is None:
        columns = df.columns.tolist()
    
    # Separar por tipo de dato
    numeric_cols = df[columns].select_dtypes(include=[np.number]).columns.tolist()
    categorical_cols = df[columns].select_dtypes(include=['object', 'category']).columns.tolist()
    
    print(f"Columnas numéricas: {len(numeric_cols)}")
    print(f"Columnas categóricas: {len(categorical_cols)}")
    
    if numeric_cols:
        n_numeric = len(numeric_cols)
        n_rows = (n_numeric + 2) // 3
        fig, axes = plt.subplots(n_rows, 3, figsize=(18, n_rows * 4))
        
        if n_rows == 1:
            axes = axes.reshape(1, -1)
        
        fig.suptitle('Distribución de Columnas Numéricas')
        
        for i, col in enumerate(numeric_cols):
            row = i // 3
            col_ax = i % 3
            ax = axes[row, col_ax]
            
            df[col].hist(bins=30, ax=ax, color='lightblue', alpha=0.7, edgecolor='black')
            ax.set_title(f'{col}\n(Únicos: {df[col].nunique()})', fontweight='bold')
            ax.set_ylabel('Frecuencia')
            ax.grid(True, alpha=0.3)
        
        # Ocultar ejes vacíos
        for i in range(len(numeric_cols), n_rows * 3):
            row = i // 3
            col_ax = i % 3
            axes[row, col_ax].set_visible(False)
        
        plt.tight_layout()
        plt.show()
    
    # Graficar categóricas
    if categorical_cols:
        n_categorical = len(categorical_cols)
        n_rows = (n_categorical + 2) // 3
        fig, axes = plt.subplots(n_rows, 3, figsize=(18, n_rows * 4))
        
        if n_rows == 1:
            axes = axes.reshape(1, -3)
        
        fig.suptitle('Distribución de Columnas Categóricas')
        
        for i, col in enumerate(categorical_cols):
            row = i // 3
            col_ax = i % 3
            ax = axes[row, col_ax]
            
            value_counts = df[col].value_counts().head(10)  # Top 10
            bars = ax.bar(range(len(value_counts)), value_counts.values, 
                         color=plt.cm.Pastel1(i / len(categorical_cols)), alpha=0.7)
            
            ax.set_xticks(range(len(value_counts)))
            ax.set_xticklabels([str(x)[:10] + '...' if len(str(x)) > 10 else str(x) 
                              for x in value_counts.index], 
                             rotation=45, ha='right', fontsize=8)
            
            # Añadir valores en barras
            for bar in bars:
                height = bar.get_height()
                ax.text(bar.get_x() + bar.get_width()/2., height,
                       f'{int(height)}', ha='center', va='bottom', fontsize=7)
            
            ax.set_title(f'{col}\n(Únicos: {df[col].nunique()})', fontweight='bold')
            ax.grid(True, alpha=0.3)
        
        # Ocultar ejes vacíos
        for i in range(len(categorical_cols), n_rows * 3):
            row = i // 3
            col_ax = i % 3
            axes[row, col_ax].set_visible(False)
        
        plt.tight_layout()
        plt.show()

In [ ]:
quick_unique_count(df[columns_high_missing_amount])

In [ ]:
plot_by_dtype_subplots(df, columns_high_missing_amount)

In [ ]:
quick_unique_count(df[columns_medium_missing_amount])

In [ ]:
plot_by_dtype_subplots(df, columns_medium_missing_amount)

### Imputacion por uso de simple imputerss

_Si bien tienen un alto valor de nulls, tiene sentido plantear que son valores nulls por default, por lo tanto vamos a utilizar imputacion por medio de valores de moda y otros por valores `0`_

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [ ]:
# para el caso de food_&_beverages
imputer = IterativeImputer(max_iter=10, random_state=0)
imputed_values = imputer.fit_transform(df[['food_&_beverages']])

df['food_&_beverages'] = np.round(imputed_values).clip(0, 5)

print("Valores únicos después de imputación:", df['food_&_beverages'].unique())

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
columns_to_impute_numeric = ["inflight_entertainment","wifi_&_connectivity"]
columns_to_impute_categoric = ["aircraft","trip_verified"]

In [ ]:
# Imputador para numéricas con valor constante 0
numeric_imputer = SimpleImputer(strategy='constant', fill_value=0)
df[columns_to_impute_numeric] = numeric_imputer.fit_transform(df[columns_to_impute_numeric])


# Imputador para categóricas con la moda
categoric_imputer = SimpleImputer(strategy='most_frequent')
df[columns_to_impute_categoric] = categoric_imputer.fit_transform(df[columns_to_impute_categoric])

_date values_

In [ ]:
df["date_flown"] = pd.to_datetime(df["date_flown"], format = "%B %Y", errors="coerce")                      

In [ ]:
# Crear características temporales antes de imputar
df['year'] = df['date_flown'].dt.year
df['month'] = df['date_flown'].dt.month
df['quarter'] = df['date_flown'].dt.quarter

# Imputar la fecha principal con moda
mode_date = df['date_flown'].mode()[0] if not df['date_flown'].mode().empty else pd.Timestamp('2019-09-01')
df['date_flown'].fillna(mode_date, inplace=True)

# Verificar resultado
print(f"Valores nulos restantes: {df['date_flown'].isnull().sum()}")
print(f"Rango de fechas: {df['date_flown'].min()} to {df['date_flown'].max()}")

df.drop(columns=['year','month','quarter'],inplace=True)


In [ ]:

constant_columns = ["ground_service" ,"overall_rating"] 

iterative_columns = ["cabin_staff_service", "seat_comfort"]

In [ ]:
imputer = IterativeImputer(max_iter=10, random_state=0)
imputed_values = imputer.fit_transform(df[constant_columns])


In [ ]:
# Imputador para numéricas con valor constante 0
numeric_imputer = SimpleImputer(strategy='constant', fill_value=0)
df[constant_columns] = numeric_imputer.fit_transform(df[constant_columns])

In [ ]:
categorical_columns = ["origin","destination","type_of_traveller"]

In [ ]:
# Encontrar la combinación origen-destino más frecuente
route_mode = df.groupby(['origin', 'destination']).size().idxmax()

# Imputar pares de forma coordinada
mask = df['origin'].isna() & df['destination'].isna()
df.loc[mask, 'origin'] = route_mode[0]
df.loc[mask, 'destination'] = route_mode[1]

df['origin'] = df['origin'].fillna('Unknown')
df['destination'] = df['destination'].fillna('Unknown')


In [ ]:
# uso de la moda para la imputacion
imputed_values = SimpleImputer(strategy='most_frequent').fit_transform(df[['type_of_traveller']])
df['type_of_traveller'] = imputed_values[:, 0]

In [ ]:
analyze_missing_values(df)

In [ ]:
df.info()

In [ ]:
plot_by_dtype_subplots(df, df.columns)

## Hipotesis 1

In [ ]:
columns_corr =[col.lower().replace(" ", "_").replace("(", "").replace(")", "") for col in
['Overall Rating', 'Value For Money', 'Seat Comfort', 'Cabin Staff Service', 'Ground Service']]

columns_corr

In [ ]:
# Análisis de correlación entre value_for_money y overall_rating
plt.figure(figsize=(15, 5))

# Gráfico 1: Correlación entre value_for_money y overall_rating
plt.subplot(1, 3, 1)
correlation_matrix = df[columns_corr].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Correlación entre Ratings')

# Gráfico 2: Distribución de overall_rating vs value_for_money
plt.subplot(1, 3, 2)
sns.scatterplot(data=df, x='value_for_money', y='overall_rating', alpha=0.6)
plt.title('value_for_money vs overall_rating')
plt.xlabel('value_for_money Rating')
plt.ylabel('overall_rating')

# Gráfico 3: Comparación de importancia de factores
plt.subplot(1, 3, 3)
factors = columns_corr
correlations_with_overall = [correlation_matrix.loc['overall_rating', factor] for factor in factors]

plt.bar(factors, correlations_with_overall, color=['red', 'blue', 'green', 'orange'])
plt.title('Correlación con overall_rating')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Análisis estadístico
print("ANÁLISIS HIPÓTESIS 1")
print(f"Correlación value_for_money - overall_rating: {correlation_matrix.loc['overall_rating', 'value_for_money']:.3f}")
print(f"Correlación Seat Comfort - overall_rating: {correlation_matrix.loc['overall_rating', 'seat_comfort']:.3f}")
print(f"Correlación Cabin Staff - overall_rating: {correlation_matrix.loc['overall_rating', 'cabin_staff_service']:.3f}")

# Test de significancia
from scipy.stats import pearsonr
corr_val, p_val = pearsonr(df['value_for_money'].dropna(), 
                          df['overall_rating'].dropna())
print(f"Valor p para correlación value_for_money: {p_val:.6f}")

*El análisis revela que "Value For Money" tiene la correlación más fuerte con el Overall Rating (correlación de ~0.85), significativamente mayor que otros factores como comodidad del asiento o servicio de cabina. Esto valida la hipótesis de que en aerolíneas low-cost, la percepción de valor económico es el principal driver de satisfacción.*

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr, chi2_contingency, f_oneway
import warnings
warnings.filterwarnings('ignore')


In [ ]:

print("HIPÓTESIS 2: Evolución Temporal de la Satisfacción")

df['year_month'] = df['date_flown'].dt.to_period('M')

# Preparar datos temporales
temporal_data = df.dropna(subset=['date_flown', 'overall_rating']).copy()

monthly_stats = temporal_data.groupby('year_month').agg({
    'overall_rating': ['mean', 'count', 'std'],
    'value_for_money': 'mean',
    'cabin_staff_service': 'mean'
}).round(3)

monthly_stats.columns = ['rating_mean', 'review_count', 'rating_std', 
                        'value_mean', 'staff_mean']
monthly_stats = monthly_stats[monthly_stats['review_count'] >= 5]  # Filtrar meses con suficientes datos

plt.figure(figsize=(15, 10))

# Gráfico 1: Evolución del rating promedio
plt.subplot(4, 1, 1)
plt.plot(monthly_stats.index.astype(str), monthly_stats['rating_mean'], 
         marker='o', linewidth=2, markersize=4, color='blue', alpha=0.7)

plt.fill_between(monthly_stats.index.astype(str), 
                monthly_stats['rating_mean'] - monthly_stats['rating_std'],
                monthly_stats['rating_mean'] + monthly_stats['rating_std'],
                alpha=0.2, color='blue')
plt.title('Evolución del Rating Promedio Mensual', fontweight='bold')
plt.xticks(rotation=90, ha='right')
plt.ylabel('Overall Rating Promedio')
plt.grid(True, alpha=0.3)

# Gráfico 2: Evolución comparada de factores
plt.subplot(4, 1, 2)
plt.plot(monthly_stats.index.astype(str), monthly_stats['rating_mean'], 
         marker='o', label='Overall Rating', linewidth=2)
plt.plot(monthly_stats.index.astype(str), monthly_stats['value_mean'], 
         marker='s', label='Value for Money', linewidth=2)
plt.plot(monthly_stats.index.astype(str), monthly_stats['staff_mean'], 
         marker='^', label='Cabin Staff', linewidth=2)
plt.title('Evolución Comparada de Factores Clave', fontweight='bold')
plt.xticks(rotation=90, ha='right')
plt.ylabel('Rating Promedio')
plt.legend()
plt.grid(True, alpha=0.3)


plt.subplot(4, 1, 3)
plt.bar(monthly_stats.index.astype(str), monthly_stats['review_count'], 
        color='green', alpha=0.7)
plt.title('Número de Reseñas por Mes', fontweight='bold')
plt.xticks(rotation=90, ha='right')
plt.ylabel('Cantidad de Reseñas')

plt.subplot(4, 1, 4)
temporal_data['year'] = temporal_data['date_flown'].dt.year
yearly_boxplot = temporal_data[temporal_data['year'] >= 2020]  # Filtrar años recientes
sns.boxplot(data=yearly_boxplot, x='year', y='overall_rating')
plt.title('Distribución de Ratings por Año', fontweight='bold')
plt.xlabel('Año')
plt.ylabel('Overall Rating')

plt.tight_layout()
plt.show()

# Análisis estadístico Hipótesis 2
print("\n ANÁLISIS ESTADÍSTICO HIPÓTESIS 2")
print("Tendencia temporal del rating promedio:")
print(f"Rating promedio inicial: {monthly_stats['rating_mean'].iloc[0]:.2f}")
print(f"Rating promedio final: {monthly_stats['rating_mean'].iloc[-1]:.2f}")
print(f"Diferencia: {monthly_stats['rating_mean'].iloc[-1] - monthly_stats['rating_mean'].iloc[0]:.2f}")

# Test de correlación temporal
monthly_stats_reset = monthly_stats.reset_index()
monthly_stats_reset['time_index'] = range(len(monthly_stats_reset))
time_corr, time_p = pearsonr(monthly_stats_reset['time_index'], 
                            monthly_stats_reset['rating_mean'])
print(f"Correlación temporal (rating vs tiempo): {time_corr:.3f}, p-value: {time_p:.4f}")


In [ ]:
print("\n2. HIPÓTESIS TEMPORAL:")
if abs(time_corr) > 0.3 and time_p < 0.05:
    trend = "mejorado" if time_corr > 0 else "deteriorado"
    print(f"CONFIRMADA: Satisfacción ha {trend} significativamente en el tiempo")
else:
    print("NO CONFIRMADA: No hay tendencia temporal significativa")

In [ ]:
print("HIPÓTESIS 3: Vulnerabilidad del Segmento de Pasajeros")

# Preparar datos por tipo de viajero
traveler_data = df.dropna(subset=['type_of_traveller', 'overall_rating']).copy()

# Consolidar categorías similares
traveler_mapping = {
    'Family Leisure': 'Family',
    'Couple Leisure': 'Couple', 
    'Solo Leisure': 'Solo',
    'Business': 'Business'
}
traveler_data['traveler_type'] = traveler_data['type_of_traveller'].map(traveler_mapping)
traveler_data = traveler_data.dropna(subset=['traveler_type'])

plt.figure(figsize=(15, 10))

# Gráfico 1: Distribución de ratings por tipo de viajero
plt.subplot(2, 2, 1)
sns.boxplot(data=traveler_data, x='traveler_type', y='overall_rating', 
            order=['Family', 'Couple', 'Solo', 'Business'])
plt.title('Distribución de Ratings por Tipo de Viajero', fontweight='bold')
plt.xlabel('Tipo de Viajero')
plt.ylabel('Overall Rating')

# Gráfico 2: Rating promedio por tipo de viajero
plt.subplot(2, 2, 2)
traveler_stats = traveler_data.groupby('traveler_type').agg({
    'overall_rating': ['mean', 'count', 'std'],
    'value_for_money': 'mean',
    'recommended': lambda x: (x == 'yes').mean()
}).round(3)

traveler_stats.columns = ['rating_mean', 'count', 'rating_std', 'value_mean', 'recommend_rate']
traveler_stats = traveler_stats.sort_values('rating_mean')

colors = ['red' if idx in ['Family', 'Couple'] else 'skyblue' for idx in traveler_stats.index]
plt.bar(traveler_stats.index, traveler_stats['rating_mean'], 
        color=colors, alpha=0.7, yerr=traveler_stats['rating_std'], capsize=5)
plt.title('Rating Promedio por Tipo de Viajero', fontweight='bold')
plt.ylabel('Overall Rating Promedio')

# Añadir valores en las barras
for i, (idx, row) in enumerate(traveler_stats.iterrows()):
    plt.text(i, row['rating_mean'] + 0.1, f'{row["rating_mean"]:.2f}', 
             ha='center', va='bottom', fontweight='bold')

# Gráfico 3: Tasa de recomendación por tipo de viajero
plt.subplot(2, 2, 3)
colors_rec = ['red' if idx in ['Family', 'Couple'] else 'skyblue' for idx in traveler_stats.index]
plt.bar(traveler_stats.index, traveler_stats['recommend_rate'] * 100, 
        color=colors_rec, alpha=0.7)
plt.title('Tasa de Recomendación por Tipo de Viajero', fontweight='bold')
plt.ylabel('Tasa de Recomendación (%)')

# Añadir valores en las barras
for i, (idx, row) in enumerate(traveler_stats.iterrows()):
    plt.text(i, row['recommend_rate'] * 100 + 1, f'{row["recommend_rate"]*100:.1f}%', 
             ha='center', va='bottom', fontweight='bold')

# Gráfico 4: Análisis de value_for_money por segmento
plt.subplot(2, 2, 4)
sns.boxplot(data=traveler_data, x='traveler_type', y='value_for_money',
            order=['Family', 'Couple', 'Solo', 'Business'])
plt.title('Percepción de Value for Money por Segmento', fontweight='bold')
plt.xlabel('Tipo de Viajero')
plt.ylabel('Value for Money Rating')

plt.tight_layout()
plt.show()

# Análisis estadístico Hipótesis 3
print("\nANÁLISIS ESTADÍSTICO HIPÓTESIS 3")
print("Estadísticas por tipo de viajero:")
print(traveler_stats)

# Test ANOVA para diferencias entre grupos
groups = [traveler_data[traveler_data['traveler_type'] == traveler]['overall_rating'] 
          for traveler in ['Family', 'Couple', 'Solo', 'Business']]

f_stat, p_val_anova = f_oneway(*groups)
print(f"\nTest ANOVA - F-statistic: {f_stat:.3f}, p-value: {p_val_anova:.6f}")

# Comparación específica Family vs Business
from scipy.stats import ttest_ind
family_ratings = traveler_data[traveler_data['traveler_type'] == 'Family']['overall_rating']
business_ratings = traveler_data[traveler_data['traveler_type'] == 'Business']['overall_rating']
t_stat, p_val_ttest = ttest_ind(family_ratings, business_ratings, equal_var=False)
print(f"Test t (Family vs Business): t-statistic = {t_stat:.3f}, p-value = {p_val_ttest:.6f}")

print(f"\nDiferencia promedio Family-Business: {family_ratings.mean() - business_ratings.mean():.3f}")


print("\n3. HIPÓTESIS DE VULNERABILIDAD DEL SEGMENTO:")
if family_ratings.mean() < business_ratings.mean() and p_val_ttest < 0.05:
    print("CONFIRMADA: Familias muestran significativamente menor satisfacción")
else:
    print("NO CONFIRMADA: No hay diferencia significativa entre segmentos")



¡Excelente! Con los resultados reales, aquí están las conclusiones corregidas:

## **CONCLUSIÓN HIPÓTESIS 2: "Hipótesis Temporal"**

### **SATISFACCIÓN HA DETERIORADO**

**Hallazgos Críticos:**

1. **Tendencia de Deterioro Significativo**
   - La satisfacción ha mostrado un **deterioro estadísticamente significativo** en el tiempo
   - Contradice la percepción inicial de mejora moderada
   - Indica **problemas estructurales o operativos** persistentes

2. **Implicaciones Alarmantes**
   - El deterioro temporal sugiere que las **iniciativas de mejora no han sido efectivas**
   - Posible **erosión de la ventaja competitiva** basada en precio
   - **Aumento de expectativas del consumidor** no satisfechas

3. **Urgencia de Intervención**
   - Necesidad de **revisión profunda** de operaciones y servicio
   - Posible **fatiga de marca** en el modelo low-cost
   - **Brecha creciente** vs competidores que mejoran servicio

---

## **"Hipótesis de Vulnerabilidad del Segmento de Pasajeros"**

### **NO CONFIRMADA - PATRÓN DIFERENTE AL ESPERADO**

**Hallazgos Contrarios a la Hipótesis:**

1. **Jerarquía Invertida de Satisfacción**
   ```
   Solo Leisure (4.46) > Couple Leisure (4.42) > Business (3.93) > Family (3.64)
   ```

2. **Resultados Estadísticos Clave:**
   - **ANOVA significativo** (p = 0.0017):  **HAY diferencias entre grupos**
   - **Test t Family vs Business NO significativo** (p = 0.421): **NO hay diferencia específica entre estos dos grupos**
   - **Las diferencias reales** están entre Solo/Couple vs Family/Business

3. **Nuevos Insights Reveladores:**

   **Segmento MÁS Satisfecho: SOLO LEISURE (4.46)**
   - Viajeros individuales son los **más contentos** con Ryanair
   - Probablemente valoran **autonomía, precio, flexibilidad**
   - **Segmento objetivo ideal** - alto satisfaction + volumen

   **Segmento ESTABLE: COUPLE LEISURE (4.42)**
   - Segundos más satisfechos
   - Balance entre experiencia compartida y pragmatismo
   - **Base sólida** del negocio

   **Segmentos PROBLEMÁTICOS:**
   - **BUSINESS (3.93)**: Expectativas de servicio no cumplidas
   - **FAMILY (3.64)**: Desafíos logísticos y de espacio

4. **Análisis de la NO Confirmación:**
   - La hipótesis original **sobrestimó** la vulnerabilidad familiar vs business
   - **Ambos segmentos (Family y Business)** muestran baja satisfacción
   - El **verdadero diferenciador** es viajero individual vs grupal/corporativo

**NUEVA HIPÓTESIS EMERGENTE:**
**"Los viajeros individuales (Solo/Couple Leisure) son significativamente más satisfechos que los viajeros con restricciones grupales/corporativas (Family/Business)"**

**Implicaciones Estratégicas Corregidas:**

1. **Reenfocar Esfuerzos de Mejora:**
   - Priorizar **Business Travelers** (brecha grande vs expectativas)
   - **Revisar políticas familiares** (aunque no son los únicos problemáticos)

2. **Fortalecer Ventaja Competitiva:**
   - **Capitalizar alta satisfacción** en segmento Solo Leisure
   - Desarrollar **ofertas específicas** para viajeros individuales

3. **Comunicación Segmentada:**
   - Mensajes diferentes para **viajeros individuales** vs **grupos/empresas**
   - Gestionar expectativas para **segmentos corporativos**


*Importante* : No tengo datos numericos, por lo tanto la busqueda de outliers no es necesario

---

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr
import warnings
warnings.filterwarnings('ignore')


In [ ]:

# Configuración
plt.style.use('default')
sns.set_palette("viridis")

# Preparar datos
df_clean = df.copy()

# Seleccionar variables numéricas para análisis de correlación
numeric_columns = [
    'overall_rating', 'seat_comfort', 'cabin_staff_service', 
    'food_&_beverages', 'ground_service', 'value_for_money',
    'inflight_entertainment', 'wifi_&_connectivity'
]

# Filtrar solo las columnas que existen en el dataset
available_columns = [col for col in numeric_columns if col in df_clean.columns]
df_corr = df_clean[available_columns].copy()

print("ANÁLISIS DE COLINEALIDAD Y CORRELACIÓN")
print(f"Variables analizadas: {available_columns}")
print(f"Tamaño de muestra: {len(df_corr)} registros\n")

# Calcular matriz de correlación
correlation_matrix = df_corr.corr().round(3)
correlation_matrix

In [ ]:
plt.figure(figsize=(16, 14))

# Heatmap 1: Matriz de correlación completa
plt.subplot(2, 2, 1)
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
heatmap = sns.heatmap(correlation_matrix, annot=True, cmap='RdBu_r', center=0,
                     square=True, fmt='.3f', mask=mask, 
                     cbar_kws={'shrink': 0.8}, annot_kws={'size': 10})
plt.title('MATRIZ DE CORRELACIÓN - TODAS LAS VARIABLES\n(Pearson Correlation)', 
          fontsize=14, fontweight='bold', pad=20)

# Heatmap 2: Correlaciones solo con overall_rating
plt.subplot(2, 2, 2)
corr_with_target = correlation_matrix[['overall_rating']].drop('overall_rating')
corr_with_target_sorted = corr_with_target.sort_values('overall_rating', ascending=False)

sns.heatmap(corr_with_target_sorted, annot=True, cmap='viridis', 
            fmt='.3f', cbar_kws={'shrink': 0.8})
plt.title('CORRELACIÓN CON OVERALL_RATING\n(Ordenado por importancia)', 
          fontsize=14, fontweight='bold', pad=20)
plt.ylabel('Variables')

plt.subplot(2, 2, 3)

# Crear matriz de colinealidad (correlaciones entre predictores, excluyendo target)
predictors_matrix = correlation_matrix.drop('overall_rating').drop('overall_rating', axis=1)

# Resaltar correlaciones altas entre predictores (potencial colinealidad)
high_collinearity_mask = (np.abs(predictors_matrix) > 0.7) & (np.abs(predictors_matrix) < 1.0)

sns.heatmap(predictors_matrix, annot=True, cmap='YlOrRd', 
            fmt='.3f', cbar_kws={'shrink': 0.8},
            mask=np.eye(len(predictors_matrix)))  # Mask diagonal
plt.title('COLINEALIDAD ENTRE PREDICTORES\n(Correlaciones > 0.7 resaltadas)', 
          fontsize=14, fontweight='bold', pad=20)


plt.tight_layout()
plt.show()


In [ ]:
# Transformar recommended a numérico
df['recommended_numeric'] = df['recommended'].map({'yes': 1, 'no': 0})

# Seleccionar solo columnas numéricas para el análisis de correlación
numeric_columns = [
    'overall_rating', 'seat_comfort', 'cabin_staff_service', 
    'food_&_beverages', 'ground_service', 'value_for_money', 
    'inflight_entertainment', 'wifi_&_connectivity', 'recommended_numeric'
]

# Calcular matriz de correlación
correlation_matrix = df[numeric_columns].corr()

# Obtener correlaciones con recommended
recommended_correlations = correlation_matrix['recommended_numeric'].sort_values(ascending=False)

print("CORRELACIONES CON RECOMMENDED (ordenadas por importancia):")
for feature, corr in recommended_correlations.items():
    if feature != 'recommended_numeric':
        print(f"{feature:25} : {corr:+.4f}")


In [ ]:
# También mostrar correlación entre features para detectar multicolinealidad
print("MATRIZ DE CORRELACIÓN COMPLETA:")
correlation_matrix.round(3)

In [ ]:
plt.rcParams['figure.figsize'] = (16, 9)

# Transformar recommended a numérico
df['recommended_numeric'] = df['recommended'].map({'yes': 1, 'no': 0})

# Seleccionar solo columnas numéricas para el análisis de correlación
numeric_columns = [
    'overall_rating', 'seat_comfort', 'cabin_staff_service', 
    'food_&_beverages', 'ground_service', 'value_for_money', 
    'inflight_entertainment', 'wifi_&_connectivity', 'recommended_numeric'
]

# Calcular matriz de correlación
correlation_matrix = df[numeric_columns].corr()

# Crear máscara para el triángulo superior
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))

# Crear el heatmap
plt.figure(figsize=(16, 9))
heatmap = sns.heatmap(correlation_matrix, 
                    mask=mask,
                    annot=True, 
                    cmap='RdBu_r', 
                    center=0,
                    fmt='.3f',
                    square=True,
                    cbar_kws={'shrink': 0.8})

# Mejorar el título y formato
plt.title('MATRIZ DE CORRELACIÓN - PREDICCIÓN DE RECOMENDACIÓN', 
         fontsize=16, fontweight='bold', pad=20)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)

# Resaltar la fila de recommended
recommended_idx = list(correlation_matrix.columns).index('recommended_numeric')
for i, label in enumerate(heatmap.get_yticklabels()):
    if i == recommended_idx:
        label.set_fontweight('bold')
        label.set_color('darkred')

for i, label in enumerate(heatmap.get_xticklabels()):
    if i == recommended_idx:
        label.set_fontweight('bold')
        label.set_color('darkred')

plt.tight_layout()
plt.show()

# Heatmap enfocado solo en correlaciones con recommended
plt.figure(figsize=(16, 9))
recommended_corrs = correlation_matrix['recommended_numeric'].drop('recommended_numeric')
recommended_corrs_sorted = recommended_corrs.sort_values(ascending=False)

# Crear gráfico de barras para correlaciones
colors = ['#2E8B57' if x > 0.5 else '#FF6B6B' if x > 0.3 else '#FFA500' for x in recommended_corrs_sorted]

plt.barh(range(len(recommended_corrs_sorted)), recommended_corrs_sorted.values, color=colors)
plt.yticks(range(len(recommended_corrs_sorted)), recommended_corrs_sorted.index)
plt.xlabel('Coeficiente de Correlación')
plt.title('CORRELACIÓN CON RECOMMENDED', fontsize=14, fontweight='bold')
plt.grid(axis='x', alpha=0.3)

# Añadir valores en las barras
for i, v in enumerate(recommended_corrs_sorted.values):
    plt.text(v + 0.01, i, f'{v:.3f}', va='center', fontweight='bold')

plt.tight_layout()
plt.show()

# Mostrar correlaciones en formato tabla
print("CORRELACIONES CON RECOMMENDED (ordenadas por importancia):")

for feature, corr in recommended_corrs_sorted.items():
    strength = "MUY FUERTE" if abs(corr) > 0.7 else "FUERTE" if abs(corr) > 0.5 else "MODERADA" if abs(corr) > 0.3 else "DÉBIL"
    print(f"{feature:25} : {corr:+.4f} ({strength})")

# RECOMENDACIÓN CONCLUSIVA - FEATURES PARA MODELO DE ML

## **FEATURES PRINCIPALES (IMPLEMENTAR SÍ O SÍ)**

### **1. `overall_rating`** 
- **Correlación: +0.9050** (MUY FUERTE)
- **Impacto**: **CRÍTICO** - Explica el 90.5% de la variabilidad en las recomendaciones
- **Recomendación**: **INCLUIR COMO FEATURE PRINCIPAL**

### **2. `value_for_money`**
- **Correlación: +0.8526** (MUY FUERTE)
- **Impacto**: **ALTO** - La relación calidad-precio es fundamental
- **Recomendación**: **INCLUIR COMO FEATURE ESENCIAL**

### **3. `cabin_staff_service`**
- **Correlación: +0.7218** (MUY FUERTE)
- **Impacto**: **ALTO** - El servicio de la tripulación es clave
- **Recomendación**: **INCLUIR COMO FEATURE PRIMARIO**

### **4. `seat_comfort`** 
- **Correlación: +0.6631** (FUERTE)
- **Impacto**: **MEDIO-ALTO** - Comodidad física importante
- **Recomendación**: **INCLUIR COMO FEATURE SECUNDARIO**

---

##  **FEATURES OPCIONALES (CONSIDERAR SEGÚN COMPLEJIDAD)**

### **5. `food_&_beverages`** & **6. `ground_service`** 
- **Correlación: ~+0.417** (MODERADA)
- **Impacto**: **MEDIO** - Contribuyen pero no son decisivos
- **Recomendación**: **INCLUIR SI SE BUSCA MÁXIMA PRECISIÓN**

---

##  **FEATURES A DESCARTAR**

### **7. `inflight_entertainment`** & **8. `wifi_&_connectivity`** 
- **Correlación: ~-0.19** (DÉBIL/NEGATIVA)
- **Impacto**: **INSIGNIFICANTE** - Prácticamente no influyen
- **Recomendación**: **EXCLUIR DEL MODELO**


# Modelo de Mahcine Learning para Prediccion


In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                           f1_score, confusion_matrix, classification_report, 
                           roc_auc_score, roc_curve)
import warnings
warnings.filterwarnings('ignore')


In [ ]:
# Transformar target a numérico
df['recommended_numeric'] = df['recommended'].map({'yes': 1, 'no': 0})

# Seleccionar features basado en el análisis de correlación
best_features = [
    'overall_rating',
    'value_for_money', 
    'cabin_staff_service',
    'seat_comfort',
    'food_&_beverages',
    'ground_service'
]


In [ ]:
X = df[best_features].copy()
y = df['recommended_numeric']


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Escalar features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Dataset: {X.shape[0]} muestras, {X.shape[1]} features")
print(f"Distribución target: {y.value_counts().to_dict()}")
print(f"Train: {X_train.shape[0]} muestras")
print(f"Test: {X_test.shape[0]} muestras")


In [ ]:
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Decision Tree': DecisionTreeClassifier(random_state=42, max_depth=5),
    'Random Forest': RandomForestClassifier(random_state=42, n_estimators=100),
    'XGBoost': XGBClassifier(random_state=42, eval_metric='logloss'),
    'LightGBM': LGBMClassifier(random_state=42, verbose=-1),
    'CatBoost': CatBoostClassifier(random_state=42, verbose=False, iterations=100)
}

In [ ]:
# Entrenar y evaluar modelos
print("ENTRENANDO Y EVALUANDO MODELOS")

results = {}
predictions = {}
y_pred_proba_dict = {}

for name, model in models.items():
    print(f"\n Entrenando {name}...")
    
    # Usar datos escalados para modelos que lo requieren
    if name in ['Logistic Regression', 'XGBoost', 'LightGBM']:
        print(f"Generando data scaled para : {name}")
        X_tr, X_te = X_train_scaled, X_test_scaled
    else:
        print(f"Generando data para : {name}")
        X_tr, X_te = X_train, X_test
    
    # Entrenar modelo
    model.fit(X_tr, y_train)
    
    # Predecir
    y_pred = model.predict(X_te)
    y_pred_proba = model.predict_proba(X_te)[:, 1]
    
    # Calcular métricas
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred_proba)
    
    # Cross validation
    cv_scores = cross_val_score(model, X_tr, y_train, cv=5, scoring='accuracy')
    
    # Guardar resultados
    results[name] = {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        'AUC': auc,
        'CV Mean': cv_scores.mean(),
        'CV Std': cv_scores.std()
    }
    
    predictions[name] = y_pred
    y_pred_proba_dict[name] = y_pred_proba
    
    print(f"{name} completado - Accuracy: {accuracy:.3f}")

# Mostrar resultados comparativos
print(" RESULTADOS COMPARATIVOS (CON CATBOOST)")

results_df = pd.DataFrame(results).T
results_df = results_df.round(4)
results_df_sorted = results_df.sort_values('Accuracy', ascending=False)


In [ ]:
results_df_sorted

In [ ]:
# Visualización de métricas
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Gráfico 1: Accuracy
results_df_sorted['Accuracy'].plot(kind='bar', ax=axes[0,0], color='skyblue', alpha=0.8)
axes[0,0].set_title('Accuracy por Modelo')
axes[0,0].set_ylabel('Accuracy')
axes[0,0].tick_params(axis='x', rotation=45)
for i, v in enumerate(results_df_sorted['Accuracy']):
    axes[0,0].text(i, v + 0.01, f'{v:.3f}', ha='center', va='bottom')

# Gráfico 2: AUC
results_df_sorted['AUC'].plot(kind='bar', ax=axes[0,1], color='lightcoral', alpha=0.8)
axes[0,1].set_title('AUC Score por Modelo')
axes[0,1].set_ylabel('AUC')
axes[0,1].tick_params(axis='x', rotation=45)
for i, v in enumerate(results_df_sorted['AUC']):
    axes[0,1].text(i, v + 0.01, f'{v:.3f}', ha='center', va='bottom')

# Gráfico 3: F1-Score
results_df_sorted['F1-Score'].plot(kind='bar', ax=axes[0,2], color='lightgreen', alpha=0.8)
axes[0,2].set_title('F1-Score por Modelo')
axes[0,2].set_ylabel('F1-Score')
axes[0,2].tick_params(axis='x', rotation=45)
for i, v in enumerate(results_df_sorted['F1-Score']):
    axes[0,2].text(i, v + 0.01, f'{v:.3f}', ha='center', va='bottom')

# Gráfico 4: Cross Validation
cv_means = results_df_sorted['CV Mean']
cv_stds = results_df_sorted['CV Std']
axes[1,0].bar(cv_means.index, cv_means.values, yerr=cv_stds.values, 
             capsize=5, alpha=0.7, color='orange')
axes[1,0].set_title('Cross Validation Accuracy (5-fold)')
axes[1,0].set_ylabel('Accuracy')
axes[1,0].tick_params(axis='x', rotation=45)
for i, v in enumerate(cv_means.values):
    axes[1,0].text(i, v + 0.01, f'{v:.3f}', ha='center', va='bottom')

# Gráfico 5: Matriz de confusión del mejor modelo
best_model_name = results_df['Accuracy'].idxmax()
best_y_pred = predictions[best_model_name]
cm = confusion_matrix(y_test, best_y_pred)

sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[1,1],
            xticklabels=['No Recomienda', 'Recomienda'],
            yticklabels=['No Recomienda', 'Recomienda'])
axes[1,1].set_title(f'Matriz de Confusión - {best_model_name}\nAccuracy: {results_df.loc[best_model_name, "Accuracy"]:.3f}')


plt.tight_layout()
plt.show()


In [ ]:
# Curvas ROC en subplots individuales
print("CURVAS ROC - TODOS LOS MODELOS")

# Calcular el número de filas y columnas para los subplots
n_models = len(models)
n_cols = 3  # Puedes ajustar este número según prefieras
n_rows = (n_models + n_cols - 1) // n_cols  # División redondeada hacia arriba

fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
axes = axes.flatten() if n_rows > 1 or n_cols > 1 else [axes]  # Aplanar el array de axes

colors = ['blue', 'green', 'red', 'purple', 'orange', 'brown', 'pink', 'gray', 'olive', 'cyan']

for i, (name, color) in enumerate(zip(models.keys(), colors)):
    if i < len(axes):  # Asegurarse de que no excedemos el número de subplots
        # Calcular métricas ROC para el modelo actual
        fpr, tpr, _ = roc_curve(y_test, y_pred_proba_dict[name])
        auc_score = roc_auc_score(y_test, y_pred_proba_dict[name])
        
        # Graficar en el subplot individual
        axes[i].plot(fpr, tpr, label=f'{name} (AUC = {auc_score:.3f})', 
                    linewidth=2.5, color=color, alpha=0.8)
        axes[i].plot([0, 1], [0, 1], 'k--', alpha=0.5, 
                    label='Clasificador Aleatorio', linewidth=2)
        
        # Configurar el subplot individual
        axes[i].set_xlabel('Tasa de Falsos Positivos (FPR)', fontsize=10)
        axes[i].set_ylabel('Tasa de Verdaderos Positivos (TPR)', fontsize=10)
        axes[i].set_title(f'Curva ROC - {name}', fontsize=12, fontweight='bold')
        axes[i].legend(fontsize=9)
        axes[i].grid(alpha=0.3)
        axes[i].set_xlim([0.0, 1.0])
        axes[i].set_ylim([0.0, 1.05])

# Ocultar los subplots vacíos si los hay
for i in range(len(models), len(axes)):
    axes[i].set_visible(False)

plt.tight_layout()
plt.show()


In [ ]:
# 🔍 IMPORTANCIA DE FEATURES - COMPARATIVO
print("IMPORTANCIA DE FEATURES - COMPARATIVO")

fig, axes = plt.subplots(2, 3, figsize=(20, 12))
all_models = ['Random Forest', 'XGBoost', 'LightGBM', 'CatBoost', 'Logistic Regression']

for i, model_name in enumerate(all_models):
    model = models[model_name]
    
    # Determinar si es modelo ensemble (usa feature_importances_) o lineal (usa coef_)
    if hasattr(model, 'feature_importances_'):
        # Para modelos ensemble
        feature_importance = pd.DataFrame({
            'feature': best_features,
            'importance': model.feature_importances_
        }).sort_values('importance', ascending=True)
        
        title = f'Feature Importance - {model_name}'
        xlabel = 'Importancia'
        
    elif hasattr(model, 'coef_'):
        # Para Logistic Regression - usar valores absolutos de coeficientes
        importance_values = np.abs(model.coef_[0])
        feature_importance = pd.DataFrame({
            'feature': best_features,
            'importance': importance_values
        }).sort_values('importance', ascending=True)
        
        title = f'Coeficientes (abs) - {model_name}'
        xlabel = '|Coeficiente|'
    
    # Crear el gráfico
    row, col = i//3, i%3
    ax = axes[row, col]
    bars = ax.barh(feature_importance['feature'], feature_importance['importance'], 
                  color=plt.cm.viridis(np.linspace(0, 1, len(best_features))))
    ax.set_title(title, fontweight='bold')
    ax.set_xlabel(xlabel)
    
    # Añadir valores en las barras
    for bar in bars:
        width = bar.get_width()
        ax.text(width + 0.001, bar.get_y() + bar.get_height()/2, 
               f'{width:.3f}', ha='left', va='center', fontsize=8)

# Feature importance promedio (solo para modelos con feature_importances_)
ensemble_models = ['Random Forest', 'XGBoost', 'LightGBM', 'CatBoost']
avg_importance = pd.DataFrame({'feature': best_features})

for model_name in all_models:
    model = models[model_name]
    
    if hasattr(model, 'feature_importances_'):
        # Para modelos ensemble
        avg_importance[model_name] = model.feature_importances_
    elif hasattr(model, 'coef_'):
        # Para Logistic Regression - normalizar coeficientes para comparar
        coef_abs = np.abs(model.coef_[0])
        # Normalizar para que sumen 1 (similar a feature_importances_)
        coef_normalized = coef_abs / coef_abs.sum()
        avg_importance[model_name] = coef_normalized

# Calcular promedio de todos los modelos
avg_importance['Average'] = avg_importance[all_models].mean(axis=1)
avg_importance = avg_importance.sort_values('Average', ascending=True)

# Gráfico de promedio
axes[1, 2].barh(avg_importance['feature'], avg_importance['Average'],
               color=plt.cm.plasma(np.linspace(0, 1, len(best_features))))
axes[1, 2].set_title('Importancia Promedio - Todos los Modelos', fontweight='bold')
axes[1, 2].set_xlabel('Importancia Promedio')

# Añadir valores en las barras del promedio
bars_avg = axes[1, 2].containers[0]
for bar in bars_avg:
    width = bar.get_width()
    axes[1, 2].text(width + 0.001, bar.get_y() + bar.get_height()/2, 
                   f'{width:.3f}', ha='left', va='center', fontsize=8)

plt.tight_layout()
plt.show()

# Conclusiones finales:

### **Mejor Opción: CatBoost**

**Razones principales:**

1. **Mayor poder discriminativo**: CatBoost muestra el rango más amplio de importancias (0.10-0.18) comparado con otros modelos, indicando una mejor capacidad para distinguir entre características importantes y menos importantes.

2. **Consistencia en características clave**:
   - `overall_raising` y `value_for_promy` son consistentemente las dos características más importantes en todos los modelos
   - CatBoost mantiene este patrón pero con mejor separación entre características

3. **Estabilidad**: Las importancias en CatBoost están mejor distribuidas sin la superposición excesiva que se observa en algunos otros modelos.

### **Ranking de Modelos:**

1. **CatBoost** - Mejor balance y discriminación
2. **Random Forest** - Buen rango de importancias pero menos refinado
3. **XGBoost** - Similar a CatBoost pero con importancias más concentradas
4. **Logistic Regression** - Patrón similar a CatBoost pero posiblemente menos robusto

### **Características Más Importantes (Consistentes en todos los modelos):**

1. **overall_raising** - La más importante en todos los modelos
2. **value_for_promy** - Segunda en importancia consistentemente
3. **ground_service** - Tercera posición en la mayoría de modelos
4. **cabin_stuff_service** - Cuarta en importancia general

### **Recomendación Final:**

**CatBoost** sería la mejor opción porque:
- Proporciona la mejor separación entre características importantes y menos importantes
- Mantiene consistencia con los patrones encontrados en otros modelos
- Es conocido por su buen rendimiento con datos categóricos y su robustez frente al overfitting
- Las importancias bien definidas facilitan la interpretación del modelo para toma de decisiones